# TensorFlow Deep Model

Ultimately we want to work in tf to build a model, as it will give the most flexibility and we can build off of some modules we built for SQuAD. 

We'll go through as follows:
0. Set-up.
1. Read in dataset.
2. Convert dataset into format for RNN.
3. Construct vocabulary for RNN.
4. Fit TF-RNN classifier.

In [67]:
# 0. Some initial set-up.
from collections import Counter
import numpy as np
import os
import pandas as pd
import random
from tf_rnn_classifier import TfRNNClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
import tensorflow as tf
import sst
import utils

In [27]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 3133903383752161713, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11285974221
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16964517380257900619
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [5]:
data_dir = "./data/"

## Read in data

In [6]:
train = pd.read_csv(data_dir + "train.csv").fillna(' ')
test = pd.read_csv(data_dir + "test.csv").fillna(' ')
test_labels = pd.read_csv(data_dir + "test_labels.csv")

In [7]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
train_text = train['comment_text']
train_labels = train[label_cols]

## Formatting dataset for RNN

In order to format the dataset for the RNN, we want to format it so that we have a list of lists. Outer list corresponds to training examples, inner list corresponds to token within each example.

In [67]:
def build_rnn_dataset(dataset, train_frac=0.9, seed=20):    
    label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    
    num_examples = len(dataset)
    num_train = int(train_frac * num_examples)
    
    np.random.seed(seed)
    shuffled = dataset.iloc[np.random.permutation(num_examples)]
    train = shuffled.iloc[:num_train]
    dev = shuffled.iloc[num_train:]
    
    train_text, dev_text = train['comment_text'], dev['comment_text']
    train_labels, dev_labels = train[label_cols], dev[label_cols]


    X_rnn_train, X_rnn_dev = [], []
    for comment in train_text:
        X_rnn_train.append(comment.split())
    for comment in dev_text:
        X_rnn_dev.append(comment.split())
        
    Y_rnn_train, Y_rnn_dev = [], []
    for _, labels in train_labels.iterrows():
        Y_rnn_train.append(list(labels))
    for _, labels in dev_labels.iterrows():
        Y_rnn_dev.append(list(labels))
        
    return ({'train': X_rnn_train, 'dev': X_rnn_dev},
            {'train': Y_rnn_train, 'dev': Y_rnn_dev})

In [45]:
X_rnn, Y_rnn = build_rnn_dataset(train, 0.9)

In [55]:
X_rnn['train'][30][:20]

['tYOU',
 'ARE',
 'A',
 'MOTHJER',
 'FUCKER',
 'COCKSUCKER!',
 'YOU',
 'ARE',
 'A',
 'MOTHJER',
 'FUCKER',
 'COCKSUCKER!',
 'YOU',
 'ARE',
 'A',
 'MOTHJER',
 'FUCKER',
 'COCKSUCKER!',
 'YOU',
 'ARE']

In [54]:
Y_rnn['train'][30]

[1, 1, 1, 0, 1, 0]

## Get vocab for RNN

In [90]:
full_train_vocab = sst.get_vocab(X_rnn['train'])

In [91]:
print("sst_full_train_vocab has {:,} items".format(len(full_train_vocab)))

sst_full_train_vocab has 494,751 items


## Train RNN 

In [108]:
num_train = 3000

In [117]:
tf_rnn = TfRNNClassifier(
    train_vocab,
    embed_dim=50,
    hidden_dim=50,
    max_length=100,
    hidden_activation=tf.nn.tanh,
    cell_class=tf.nn.rnn_cell.LSTMCell,
    train_embedding=True,
    max_iter=50,
    eta=0.1)

In [118]:
_ = tf_rnn.fit(X_rnn['train'][:], Y_rnn['train'][:])

Iteration 50: loss: 6.6362756900489335

## Evaluate on dev set

In [131]:
tf_rnn_predictions = tf_rnn.predict(X_rnn['dev'])

In [108]:
def evaluate(labels, preds):
    """
    labels: list (of shape (num_ex, num_class))
    preds: np.array shape (num_ex, num_class) (can be probabilistic)
    """
    labels = np.array(labels)

    # Class-wise Precision/Recall/F1
    label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    for i in range(len(label_cols)):
        print("CLASS: %s" % label_cols[i])
        print(classification_report(labels[:, i], np.round(tf_rnn_predictions[:,i])))
        print()
        
    # ROC-AUC
    roc_auc = roc_auc_score(labels, preds)
    print("macro-averaged ROC-AUC score: %f" % roc_auc)
    

In [109]:
evaluate(Y_rnn_train[:num_train], tf_rnn_predictions)

CLASS: toxic
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2693
          1       1.00      1.00      1.00       307

avg / total       1.00      1.00      1.00      3000


CLASS: severe_toxic
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2962
          1       1.00      1.00      1.00        38

avg / total       1.00      1.00      1.00      3000


CLASS: obscene
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2833
          1       1.00      0.99      1.00       167

avg / total       1.00      1.00      1.00      3000


CLASS: threat
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2986
          1       1.00      1.00      1.00        14

avg / total       1.00      1.00      1.00      3000


CLASS: insult
             precision    recall  f1-score   support

      

In [105]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
for i in range(len(label_cols)):
    print("CLASS: %s" % label_cols[i])
    print(classification_report(np.array(Y_rnn_train[:num_train])[:, i], np.round(tf_rnn_predictions[:,i])))
    print()

CLASS: toxic
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2693
          1       1.00      1.00      1.00       307

avg / total       1.00      1.00      1.00      3000


CLASS: severe_toxic
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2962
          1       1.00      1.00      1.00        38

avg / total       1.00      1.00      1.00      3000


CLASS: obscene
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2833
          1       1.00      0.99      1.00       167

avg / total       1.00      1.00      1.00      3000


CLASS: threat
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      2986
          1       1.00      1.00      1.00        14

avg / total       1.00      1.00      1.00      3000


CLASS: insult
             precision    recall  f1-score   support

      